## Make test set for image 0748

Given the clean bboxes and the .tif create overlapping windows that contain any bbxos.
1. Rolling window
2. If Window bounds intersects with bbox save window

## Run test prediction and compute scores

1. Run model on previously saved test tiles ? or on the whole image?
2. 

In [9]:
from itertools import product

import geopandas as gpd
import pandas as pn
import numpy as np
import rasterio
from rasterio.plot import show
from rasterio.windows import from_bounds, 
from rasterio.plot import reshape_as_image
import matplotlib.pyplot as plt
from matplotlib import patches
from tqdm.notebook import tqdm
from shapely.geometry import Polygon, box
from shapely.ops import unary_union

import cv2
from pytorchyolo import detect, models

In [2]:
gdf = gpd.read_file("../test_bboxes/test_bbox_rev.shp").explode()  # explode to make multipolygon single polygon

In [6]:
img = "/media/biel/SharedHDD/Master/TFM/ortos/PNOA_MA_OF_ETRS89_HU31_h50_0748.tif"

In [17]:
def overlapping_windows(src, overlap, width, height):
    """"width & height not including overlap i.e requesting a 256x256 window with 
        1px overlap will return a 258x258 window (for non edge windows)"""
    offsets = product(range(0, src.meta['width'], width), range(0, src.meta['height'], height))
    for col_off, row_off in offsets:
        window = rasterio.windows.Window(
            col_off=col_off - overlap,
            row_off=row_off - overlap,
            width=width + overlap * 2,
            height=height + overlap * 2)        
        yield window

def detection_to_geo(model, img: str) -> gpd.GeoDataFrame:
    
    for img in tqdm(images):
        with rasterio.open(img) as src:
            # rasterio native shape is (bands, rows, columns) but we need (rows, columns, bands)
            # see https://rasterio.readthedocs.io/en/latest/topics/image_processing.html#imageorder
            img = reshape_as_image(src.read())
            tranform = src.transform
        boxes = detect.detect_image(model, img, conf_thres=0.1, nms_thres=0.1)
        for x1, y1, x2, y2, conf, _ in boxes:
            # transform from pixels coordinates to geografic coordinates 
            # using the affine transformation matrix
            geo_x1, geo_y1 = tranform * np.array([x1, y1])
            geo_x2, geo_y2 = tranform * np.array([x2, y2]) 
            coords = ((geo_x1, geo_y1), (geo_x2, geo_y1), (geo_x2, geo_y2), (geo_x1, geo_y2))
            data["geometry"].append(Polygon(coords))
            data["confidence"].append(conf)
    return gpd.GeoDataFrame(data)

In [18]:
model = models.load_model(
  "PyTorch-YOLOv3/config/yolov3-custom.cfg", 
  "PyTorch-YOLOv3/checkpoints/saves/yolov3_ckpt_183.pth"
)

In [22]:
with rasterio.open(img) as src:
    data = {"geometry": [], "confidence": []}
    for window in tqdm(overlapping_windows(src, 30, 452, 452)):
        img_window = reshape_as_image(src.read(window=window))
        detected_bbox = detect.detect_image(model, img_window, conf_thres=0.1, nms_thres=0.1)
        if detected_bbox.size > 0:
            # get the window transform matrix
            print("Pools detected!")
            transform = rasterio.windows.transform(window, src.transform)
            for x1, y1, x2, y2, conf, _ in detected_bbox:
                # transform from pixels coordinates to geografic coordinates 
                # using the affine transformation matrix
                geo_x1, geo_y1 = transform * np.array([x1, y1])
                geo_x2, geo_y2 = transform * np.array([x2, y2]) 
                coords = ((geo_x1, geo_y1), (geo_x2, geo_y1), (geo_x2, geo_y2), (geo_x1, geo_y2))
                data["geometry"].append(Polygon(coords))
                data["confidence"].append(conf)

0it [00:00, ?it/s]

Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools detected!
Pools de

In [24]:
results_df = gpd.GeoDataFrame(data)

In [26]:
results_df.to_file("results/test_results.shp", index=True)